In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import glob
import random
from data_loader import load_map, Note
from concurrent.futures import ProcessPoolExecutor
import json
import shutil
import os
import traceback
from data_generator import create_datasets


In [2]:
random_seed = 1470258369

random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [3]:
train_ds, val_ds = create_datasets(batch_size=128, val_split=0.1, random_seed=random_seed)

In [4]:
# # preload the dataset into cache to keep the data loading errors away from the training logs
# discard_value = [0 for v in tqdm(train_ds)]

# discard_value = [0 for v in tqdm(val_ds)]

In [5]:
import models.placement_features
import models.placement
# from importlib import reload
# reload(models.placement_features)
# reload(models.placement)

In [6]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
forgor to give concrete timings for parity and flow understanding

def make_model():
    input_prev_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    input_prev_notes = tf.keras.Input(shape=(2, 40, 25), dtype="float32")
    input_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    input_acc_prediction = tf.keras.Input(shape=(1, ), dtype="float32")
    input_speed_prediction = tf.keras.Input(shape=(1, ), dtype="float32")
    input_y_notes = tf.keras.Input(shape=(2, 40, 25), dtype="float32")

    l_prevstyle_prev_features, l_prevstyle_features, l_ystyle_prev_features, l_ystyle_features = models.placement_features.create_placement_features_model()([input_prev_audio, input_audio, input_prev_notes, input_acc_prediction, input_speed_prediction, input_y_notes])
    
    l_prevstyle_predictions = tf.keras.layers.Concatenate(axis=1)([l_prevstyle_prev_features, l_prevstyle_features])
    l_ystyle_predictions = tf.keras.layers.Concatenate(axis=1)([l_ystyle_prev_features, l_ystyle_features])
    
    placement_selection_model = models.placement.create_placement_selection_model()
    
    prevstyle_placement = placement_selection_model([l_prevstyle_predictions, input_prev_notes])
    ystyle_placement = placement_selection_model([l_ystyle_predictions, input_prev_notes])

    model = tf.keras.Model(inputs = [input_prev_audio, input_prev_notes, input_audio, input_acc_prediction, input_speed_prediction, input_y_notes], outputs = [prevstyle_placement, ystyle_placement])
    return model

In [7]:
model = make_model()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 87, 129,  0           []                               
                                 1)]                                                              
                                                                                                  
 input_3 (InputLayer)           [(None, 2, 87, 129,  0           []                               
                                 1)]                                                              
                                                                                                  
 input_2 (InputLayer)           [(None, 2, 40, 25)]  0           []                               
                                                                                            

In [8]:
# visualize the model
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True, dpi=384)

In [9]:
optimizer = tf.keras.optimizers.Adam()

In [10]:
metric_prevstyle_timings_loss = tf.keras.metrics.Mean(name="prevstyle_timings_loss")
metric_prevstyle_position_loss = tf.keras.metrics.Mean(name="prevstyle_position_loss")
metric_prevstyle_angle_loss = tf.keras.metrics.Mean(name="prevstyle_angle_loss")
metric_ystyle_timings_loss = tf.keras.metrics.Mean(name="ystyle_timings_loss")
metric_ystyle_position_loss = tf.keras.metrics.Mean(name="ystyle_position_loss")
metric_ystyle_angle_loss = tf.keras.metrics.Mean(name="ystyle_angle_loss")
metric_loss = tf.keras.metrics.Mean(name="loss")

### Custom loss
Calculates 2 separate values:
 - timing loss - simple loss based on when the notes were placed
 - positioning loss - loss based on the position and direction of the placed note, adjusted for the number of notes that appear in different positions to avoid a massive bias towards placing most commonly appearing notes

In [11]:
tf.config.run_functions_eagerly(False)

In [12]:
@tf.function
def custom_loss_style(y, predictions):
    timings = y[:, :, :, :1]
    position = y[:, :, :, 1::2]
    angle = y[:, :, :, 2::2]
    
    timings_predictions, position_predictions, angle_predictions = predictions
    
    sum_notes = tf.reduce_sum(position) + 0.000001
    
    timings_loss = tf.reduce_mean(tf.square(timings - timings_predictions)*(timings + 0.1)) * 6.9
    position_loss = tf.reduce_sum(tf.square(position - position_predictions)*(timings)*(position + 0.069)) / sum_notes * 0.5
    angle_loss = tf.reduce_sum(tf.square(tf.minimum(tf.abs(angle - angle_predictions), tf.minimum(tf.abs(angle - angle_predictions - 1), tf.abs(angle - angle_predictions + 1)))) * (position)) / sum_notes * 6.9
    
    return timings_loss, position_loss, angle_loss


@tf.function
def custom_loss(y, predictions):
    prev_notes, notes = y
    prevstyle_predictions, ystyle_predictions = predictions
    prevstyle_timings_loss, prevstyle_position_loss, prevstyle_angle_loss = custom_loss_style(notes, (prevstyle_predictions[0][:, :, 40:, :], prevstyle_predictions[1][:, :, 40:, :], prevstyle_predictions[2][:, :, 40:, :]))
    ystyle_timings_loss, ystyle_position_loss, ystyle_angle_loss = custom_loss_style(prev_notes, (ystyle_predictions[0][:, :, :40, :], ystyle_predictions[1][:, :, :40, :], ystyle_predictions[2][:, :, :40, :]))
    
    loss = (prevstyle_timings_loss + prevstyle_position_loss + prevstyle_angle_loss) + \
        (ystyle_timings_loss + ystyle_position_loss + ystyle_angle_loss) * 0.5
    
    metric_prevstyle_timings_loss(prevstyle_timings_loss)
    metric_prevstyle_position_loss(prevstyle_position_loss)
    metric_prevstyle_angle_loss(prevstyle_angle_loss)
    metric_ystyle_timings_loss(ystyle_timings_loss)
    metric_ystyle_position_loss(ystyle_position_loss)
    metric_ystyle_angle_loss(ystyle_angle_loss)
    metric_loss(loss)
    
    return loss

In [13]:
@tf.function
def train_step(model, optimizer, data):
    x1, x2, x3, x4, x5, x6, x7, x8, y = data
    
    with tf.GradientTape() as tape:
        predictions = model([x1, x2, x3, x7, x8, y], training=True)
        # with tf.device('/CPU:0'):
        loss = custom_loss([x2, y], predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))


@tf.function
def val_step(model, data):
    x1, x2, x3, x4, x5, x6, x7, x8, y = data
    
    predictions = model([x1, x2, x3, x7, x8, y], training=False)
    # with tf.device('/CPU:0'):
    loss = custom_loss([x2, y], predictions)

In [14]:
for epoch in range(0, 15):
    metric_prevstyle_timings_loss.reset_states()
    metric_prevstyle_position_loss.reset_states()
    metric_prevstyle_angle_loss.reset_states()
    metric_ystyle_timings_loss.reset_states()
    metric_ystyle_position_loss.reset_states()
    metric_ystyle_angle_loss.reset_states()
    metric_loss.reset_states()
    
    if epoch == 0:
        optimizer.learning_rate.assign(0.001)
    elif epoch == 2:
        optimizer.learning_rate.assign(0.00025)
    elif epoch == 4:
        optimizer.learning_rate.assign(0.0001)
    elif epoch == 8:
        optimizer.learning_rate.assign(0.000025)
    elif epoch == 10:
        optimizer.learning_rate.assign(0.00001)
    elif epoch == 12:
        optimizer.learning_rate.assign(0.0000025)
    
    with tqdm(train_ds.enumerate(), unit="batch") as _tqdm:
        _tqdm.set_description(f"Epoch train: {epoch}")
        for step, data in _tqdm:
            train_step(model, optimizer, data)
            _tqdm.set_postfix(
                prevstyle_timings=metric_prevstyle_timings_loss.result().numpy(), 
                prevstyle_position=metric_prevstyle_position_loss.result().numpy(), 
                prevstyle_angle=metric_prevstyle_angle_loss.result().numpy(), 
                ystyle_timings=metric_ystyle_timings_loss.result().numpy(), 
                ystyle_position=metric_ystyle_position_loss.result().numpy(), 
                ystyle_angle=metric_ystyle_angle_loss.result().numpy(), 
                loss=metric_loss.result().numpy(), 
            )
    
    metric_prevstyle_timings_loss.reset_states()
    metric_prevstyle_position_loss.reset_states()
    metric_prevstyle_angle_loss.reset_states()
    metric_ystyle_timings_loss.reset_states()
    metric_ystyle_position_loss.reset_states()
    metric_ystyle_angle_loss.reset_states()
    metric_loss.reset_states()
    
    if 'val_ds' in locals() or 'val_ds' in globals():
        with tqdm(val_ds.enumerate(), unit="batch") as _tqdm:
            _tqdm.set_description(f"Epoch val: {epoch}")
            for step, data in _tqdm:
                val_step(model, data)
                _tqdm.set_postfix(
                    prevstyle_timings=metric_prevstyle_timings_loss.result().numpy(), 
                    prevstyle_position=metric_prevstyle_position_loss.result().numpy(), 
                    prevstyle_angle=metric_prevstyle_angle_loss.result().numpy(), 
                    ystyle_timings=metric_ystyle_timings_loss.result().numpy(), 
                    ystyle_position=metric_ystyle_position_loss.result().numpy(), 
                    ystyle_angle=metric_ystyle_angle_loss.result().numpy(), 
                    loss=metric_loss.result().numpy(), 
                )

Epoch train: 0: : 0batch [00:00, ?batch/s]

Epoch train: 0: : 1977batch [11:22,  2.90batch/s, loss=0.595, prevstyle_angle=0.142, prevstyle_position=0.143, prevstyle_timings=0.246, ystyle_angle=0.0543, ystyle_position=0.0452, ystyle_timings=0.0285]
Epoch val: 0: : 234batch [01:14,  3.15batch/s, loss=0.403, prevstyle_angle=0.119, prevstyle_position=0.131, prevstyle_timings=0.142, ystyle_angle=0.0177, ystyle_position=0.00386, ystyle_timings=0.000536]
Epoch train: 1: : 1977batch [10:35,  3.11batch/s, loss=0.365, prevstyle_angle=0.109, prevstyle_position=0.127, prevstyle_timings=0.122, ystyle_angle=0.0103, ystyle_position=0.00217, ystyle_timings=0.000259]
Epoch val: 1: : 236batch [00:51,  4.58batch/s, loss=0.35, prevstyle_angle=0.108, prevstyle_position=0.121, prevstyle_timings=0.117, ystyle_angle=0.00646, ystyle_position=0.00117, ystyle_timings=0.000109]  
Epoch train: 2: : 1976batch [09:54,  3.32batch/s, loss=0.328, prevstyle_angle=0.0972, prevstyle_position=0.12, prevstyle_timings=0.108, ystyle_angle=0.00459, ystyle_position=0.000

KeyboardInterrupt: 

In [15]:
def full_validation(timing_threshhold, epoch, acc_prediction, speed_prediction):
    base_validation_path = "./validation"
    os.makedirs(base_validation_path, exist_ok=True)
    
    validation_map = "DA42AF71F4CA5AD280C3F69BCA0BD6C6D1CDA06E"
    validation_map_dir = f"{base_validation_path}/{validation_map}"
    (song_data, segment_duration), diffs = load_map(validation_map_dir)
    
    angle_to_direction = {
        180:0,
        0:1,
        90:2,
        270:3,
        135:4,
        225:5,
        45:6,
        315:7
    }
    direction_to_angle = {
        0: 180,
        1: 0,
        2: 90,
        3: 270,
        4: 135,
        5: 225,
        6: 45,
        7: 315
    }

    def get_note_angle(direction):
        return direction_to_angle[direction] / 360
    
    def get_note_direction(angle):
        angle = int(angle * 360)
        if angle % 45 > 22.5:
            angle += 45
        angle = (angle - angle % 45) % 360
        return angle_to_direction[angle]

    def validate_model(song_data, segment_duration, timing_threshhold, positioning_threshhold, intensity_1, intensity_2, acc_prediction, speed_prediction, note_pos_count):
        context_length = 1
        prediction_note_count = context_length * 40
        prediction_note_time_length = context_length / prediction_note_count

        context_steps = int(context_length / segment_duration) + 1
        step_size = context_steps
        
        generated_notes = []
        max_val_timing = 0
        max_val_positioning = 0
        
        zero_notes = 1
        one_notes = 1
        
        prev_note_segment = ([[0]*25 for i in range(prediction_note_count)], [[0]*25 for i in range(prediction_note_count)])
        prev_audio_segment = song_data[:, :context_steps, :]
        with tqdm(range(context_steps, song_data.shape[1] - context_steps, step_size)) as _tqdm:
          for i in _tqdm:
            curr_time = i * segment_duration
            
            x_context_prev_audio = prev_audio_segment
            x_context_prev_notes = prev_note_segment
            x_context_audio = song_data[:, i:i+context_steps, :]
            prevstyle_predictions, ystyle_predictions = model([np.array([x_context_prev_audio]), np.array([x_context_prev_notes]), np.array([x_context_audio]), np.array([acc_prediction + (random.random() - 0.5) * 0.1]), np.array([speed_prediction + (random.random() - 0.5) * 0.1]), np.array([[[[0]*25]*40, [[0]*25]*40]])], training=False)
            prevstyle_timings, prevstyle_position, prevstyle_angle = prevstyle_predictions
            model_timing_prediction = np.array(prevstyle_timings[0][:, 40:, :])
            model_position_prediction = np.array(prevstyle_position[0][:, 40:, :])
            model_angle_prediction = np.array(prevstyle_angle[0][:, 40:, :])

            x_context_prev_audio = x_context_audio
            prev_note_segment = ([[0]*25 for i in range(prediction_note_count)], [[0]*25 for i in range(prediction_note_count)])

            # I use them to find values that would generate a reasonable number of notes.
            # Small adjustments to the model and it's loss function can significantly shift the actual number values.
            if max_val_timing < np.max(model_timing_prediction):
                max_val_timing = np.max(model_timing_prediction)
                print(f"max_timing: {max_val_timing}")
                
                # if max_val_positioning < np.max(positioning_prediction):
                #     max_val_positioning = np.max(positioning_prediction)
                #     print(f"max_positioning: {max_val_positioning}")
            
            # for j in range(prediction_note_count):
            #      for color in range(2):
            for color in range(2):
                for (j,) in np.argsort(model_timing_prediction[color], axis=0)[-(random.randint(4, 8)):]:
                    curr_note_time = curr_time + j * prediction_note_time_length
                    prediction_timing = model_timing_prediction[color][j][0]
                    if prediction_timing < timing_threshhold:
                        continue
                    
                    prediction_positioning = model_position_prediction[color][j]
                    prediction_angle = model_angle_prediction[color][j]
                    
                    # prediction_positioning[:12] = prediction_positioning[:12] * (one_notes / (zero_notes + one_notes + 0.00001))
                    # prediction_positioning[12:] = prediction_positioning[12:] * (zero_notes / (zero_notes + one_notes + 0.00001))

                    # Place only 1 note per timing or to place many notes. More than 1 note can get super repetitive more easily, but both are of quite poor quality so far.
                    max_one_note_per_placement = True
                    if max_one_note_per_placement:
                        note_prediction_iter = np.argmax(prediction_positioning)
                        prediction_positioning_enumerated = [(note_prediction_iter, prediction_positioning[note_prediction_iter], prediction_angle[note_prediction_iter])]
                    else:
                        prediction_positioning_enumerated = [(i, note_prediction, prediction_angle[i]) for i, note_prediction in enumerate(prediction_positioning) if note_prediction > positioning_threshhold]

                    for note_prediction_iter, note_prediction, angle_prediction in prediction_positioning_enumerated:
                            line_layer = note_prediction_iter % 3
                            line_index = int(note_prediction_iter / 3) % 4
                            direction = get_note_direction(angle_prediction)
                            generated_notes.append(Note(curr_note_time, line_index, line_layer, color, direction))
                            if color == 0:
                                zero_notes += 1
                            else:
                                one_notes += 1
                            
                            prev_note_segment[color][j][0] = 1
                            prev_note_segment[color][j][1 + note_prediction_iter * 2] = 1
                            prev_note_segment[color][j][1 + note_prediction_iter * 2 + 1] = get_note_angle(direction)
                            
            if len(generated_notes) > 0:
                average_notes_per_second = len(generated_notes)/generated_notes[-1].time
            else:
                average_notes_per_second = -1
            _tqdm.set_postfix(average_notes_per_second=average_notes_per_second)
        
        generated_notes.sort(key=lambda note: note.time)
        return generated_notes

    intensity_timings_per_second = 7 # model input for number of correct timings per second
    intensity_notes_per_second = intensity_timings_per_second # model input for sum of '1's in the prediction segment. Increasing this value should result in more stacks and sliders.
    note_pos_count = 3
    generated_notes = validate_model(song_data, segment_duration, timing_threshhold=timing_threshhold, positioning_threshhold=0.45, intensity_1=intensity_timings_per_second, intensity_2=intensity_notes_per_second/20, acc_prediction=acc_prediction, speed_prediction=speed_prediction, note_pos_count=note_pos_count/10)

    if len(generated_notes) > 0:
        average_notes_per_second = len(generated_notes)/generated_notes[-1].time
    else:
        average_notes_per_second = -1
    
    with open(validation_map_dir + "/Info.dat", "rb") as f:
        info_json = json.load(f)
        bpm = info_json["_beatsPerMinute"]
        
    with open(validation_map_dir + "/ExpertStandard.dat", "rb") as f:
        diff_json = json.load(f)

    diff_json["_notes"] = [{"_time": note.time / 60 * bpm, "_lineIndex": int(note.lineIndex), "_lineLayer": int(note.lineLayer), "_type": int(note.type), "_cutDirection": int(note.direction)} for note in generated_notes]
    if len(diff_json["_notes"]) == 0:
        diff_json["_notes"] = [{"_time": 1, "_lineIndex": 0, "_lineLayer": 0, "_cutDirection": 0, "_type": 0}]
    with open(validation_map_dir + "/ExpertStandard.dat", "w") as f:
        json.dump(diff_json, f)
        
    shutil.make_archive(f"{validation_map_dir}q{epoch}q{timing_threshhold}q{average_notes_per_second}q{acc_prediction}q{speed_prediction}", 'zip', validation_map_dir)

In [16]:
full_validation(0.0 + (random.random() - 0.5)*0.1, epoch, 0.705 + (random.random() - 0.5)*0.15, 0.3 + (random.random() - 0.5)*0.4)

  1%|          | 1/169 [00:00<00:49,  3.37it/s, average_notes_per_second=8.38]

max_timing: 0.9623913168907166
max_timing: 0.9673818349838257


  2%|▏         | 3/169 [00:00<00:37,  4.40it/s, average_notes_per_second=10.5]

max_timing: 0.9744083285331726


 10%|█         | 17/169 [00:03<00:31,  4.82it/s, average_notes_per_second=11.7]

...

 25%|██▌       | 43/169 [00:08<00:24,  5.15it/s, average_notes_per_second=11.8]

max_timing: 0.9835774898529053


 42%|████▏     | 71/169 [00:14<00:19,  5.13it/s, average_notes_per_second=11.7]

max_timing: 0.9880657196044922


100%|██████████| 169/169 [00:34<00:00,  4.84it/s, average_notes_per_second=11.9]
